<a href="https://colab.research.google.com/github/kleinfossil/googlecolabfree_stable_diffusion/blob/main/ControlNet_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ControlNet for Stable Diffusion on Google Colab Free
This will setup ControlNet.
Make sure that you select Runtime/Change runtime type -> GPU

The requirements installation and the setup of the conda environment takes currently around **10 Minutes**.


In [ ]:
#@markdown ## Step 1: Setup CondaColab - expected time: 0m40s
#@markdown This cell need to run before all other cells, as it will restart the kernel. (The restart will look like a crash for unknown reasons)
#@markdown
#@markdown Only execute this cell alone. Then you you can excute all other cells.
"""
ControlNet uses a conda environment. This is quite challenging to setup in google colab. 
It is not possible to setup and activete a conda environment via iphython. 
Instead it is required to run the environment in a bash shell. This again denies options like code highlighting and so on.
Condacolab supports an easy setup of a conda environment. However this works only for the base environment. Also it can not change the python version. 
As ControlNet requires an older python version, this requirement will be just ignored. 
The following will setup the CondaColab which will also restart the colab kernel. Therefore this cell need to be executed seperatly. 
If there is anybody who has a better idea let me know. 
"""
from IPython.display import clear_output
!pip install -q condacolab
import condacolab
condacolab.install()
clear_output()

In [ ]:
#@markdown ## Step 2: Setup and Run ControlNet - expected time: 6m23s
#@markdown

# Download ControlNet Git
from IPython.display import clear_output
from IPython.utils import capture
import os

with capture.capture_output() as cap:
  if os.path.exists("/content/ControlNet"):
    %cd /content/ControlNet
    !git pull https://github.com/lllyasviel/ControlNet.git
  else:
      !git clone https://github.com/lllyasviel/ControlNet.git
print(cap)
clear_output()

"""
Adjust environment.yaml. This might become a point of failure.
If ther is any dependency error, then it is probably because of these changes.
"""
# Replace the environment name with base. condacolab works with base environment only.
!sed -i -e 's/name: control/name: base/g' /content/ControlNet/environment.yaml
# Delete the python requirement
!sed -i -e 's/  - python=3.8.5//g' /content/ControlNet/environment.yaml
# Delete cudatoolkit
!sed -i -e 's/  - cudatoolkit=11.3//g' /content/ControlNet/environment.yaml
# Clear empyt lines
!sed '/^$/d' /content/ControlNet/environment.yaml
clear_output()


# Check if condacolab is installed correctly
# TODO: need to handle the check
import condacolab
condacolab.check()

# Update the base environment and return error code
# This part alone will take about 7Minutes
!mamba env update -f /content/ControlNet/environment.yaml
clear_output()

# Now the program will install the required annotator packages from huggingface
!pip install huggingface_hub
from huggingface_hub import hf_hub_download
from distutils.dir_util import copy_tree


# Download Annotator Files
%mkdir -p /content/ControlNet_Hugging_Downloads
# TODO: discover the annotator packages automatically instead of hardcoding them.
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/body_pose_model.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/dpt_hybrid-midas-501f0c75.pt", cache_dir="/content/ControlNet_Hugging_Downloads")
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/hand_pose_model.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/mlsd_large_512_fp32.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/mlsd_tiny_512_fp32.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/network-bsds500.pth", cache_dir="/content/ControlNet_Hugging_Downloads")
hf_hub_download(repo_id="lllyasviel/ControlNet", filename="annotator/ckpts/upernet_global_small.pth", cache_dir="/content/ControlNet_Hugging_Downloads")

# Copies the Annotator Files
copy_tree("/content/ControlNet_Hugging_Downloads/models--lllyasviel--ControlNet/snapshots/703a9194e2224a498d610a8384aa7edbb6dc24e3/annotator/", "/content/ControlNet/annotator/")
!rm /content/ControlNet_Hugging_Downloads/models--lllyasviel--ControlNet/snapshots/703a9194e2224a498d610a8384aa7edbb6dc24e3/annotator/*
clear_output()

In [ ]:
#@markdown ## Step 3: Download Model and Run ControlNet - expected time: 2m46s
#@markdown This step will create after about 3 minutes an *external link*. Click on the link to open ControlNet App.
#@markdown

# This avoids that all models need to be downloaded at once. 
#@markdown Select Model (More Infos about models: https://github.com/lllyasviel/ControlNet):
model = "Fake Scribbles" #@param ["Canny Edge","M-LSD Lines","HED Boundary","User Scribbles","User Scribbles - Interactive","Fake Scribbles","Human Pose","Semantic Segmentation", "Depth", "Normal Map"]
# TODO: Add Non-Prompt Mode - Guess Mode
#@markdown You can change the model by stoping the Step 3, changing the model and starting Step 3
#@markdown 
#@markdown Creating 5 images with 20 Steps will take 1m17s

from pathlib import Path

if model == "Canny Edge":
  python_file = "gradio_canny2image.py"
  model_path = "models/control_sd15_canny.pth"
elif model == "M-LSD Lines":
  python_file = "gradio_hough2image.py"
  model_path = "models/control_sd15_mlsd.pth"
elif model == "HED Boundary":
  python_file = "gradio_hed2image.py"
  model_path = "models/control_sd15_hed.pth"
elif model == "User Scribbles":
  python_file = "gradio_scribble2image.py"
  model_path = "models/control_sd15_scribble.pth"
elif model == "User Scribbles - Interactive":
  python_file = "gradio_scribble2image_interactive.py"
  model_path = "models/control_sd15_scribble.pth"
elif model == "Fake Scribbles":
  python_file = "gradio_fake_scribble2image.py"
  model_path = "models/control_sd15_scribble.pth"
elif model == "Human Pose":
  python_file = "gradio_pose2image.py"
  model_path = "models/control_sd15_openpose.pth"
elif model == "Semantic Segmentation":
  python_file = "gradio_seg2image.py"
  model_path = "models/control_sd15_seg.pth"
elif model == "Depth":
  python_file = "gradio_depth2image.py"
  model_path = "models/control_sd15_depth.pth"
elif model == "Normal Map":
  python_file = "gradio_normal2image.py"
  model_path = "models/control_sd15_normal.pth"

"""
ControlNet does not support the share launch option out of the box.
A quick and dirty fix is to replace the launch call in the file.  
"""
!sed -i -e "s/block.launch(server_name='0.0.0.0')/block.launch(server_name='0.0.0.0', share=True)/g" /content/ControlNet/{python_file}

model_file = Path(f"/content/ControlNet/{model_path}")
if model_file.is_file():
  print("Model exists, no download")
else:
  # downloads the model
  hf_hub_download(repo_id="lllyasviel/ControlNet", filename=model_path, cache_dir="/content/ControlNet_Hugging_Downloads")
  clear_output()

  """
  Looks like google colab does not move the downloaded files just with bash cp or mv.
  Instead it creates broken links. I could only solve this with copy_tree.
  """
  # Moves files to the ControlNet dir. 
  # Execution time: 1m 42s
  # TODO: check if a model is already installed before doing this step
  # TODO: CRITICAL: change the hardcoded snapshot folder to a variable one
  print("Copy model to model folder...")
  copy_tree("/content/ControlNet_Hugging_Downloads/models--lllyasviel--ControlNet/snapshots/703a9194e2224a498d610a8384aa7edbb6dc24e3/models/", "/content/ControlNet/models/")
  !rm /content/ControlNet_Hugging_Downloads/models--lllyasviel--ControlNet/snapshots/703a9194e2224a498d610a8384aa7edbb6dc24e3/models/*
clear_output()

%cd /content/ControlNet/
!python {python_file}